# Ordering and Delivery

This notebook demonstrates ordering and download with the orders api. In this notebook, we check authentication by requesting an orders list, then we create an order for two `ortho_analytic_4b` `PSScene` images. We poll for order success then download images individually. And finally, we create, poll, and download the same order delivered as a single zip file.

Reference information can be found at [Orders API](https://developers.planet.com/apis/orders/).

In [22]:
import json
import os
import pathlib
import time

import requests
from planet import Session, DataClient, OrdersClient, Auth

## Authenticating

In [47]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PASTE_YOUR_API_KEY_HERE'
    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

In [35]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2' 

### Curl example

To check your orders list and make sure you have the permissions you need, uncomment the following line to run `curl`

In [36]:
# !curl -L -H "Authorization: api-key $API_KEY" $orders_url

### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/apis/orders/reference/#tag/Orders/operation/listOrders) API reference.

In [49]:
response = requests.get(orders_url, auth=session.auth)
response

<Response [200]>

Now we will list the orders we have created thus far. Your list may be empty if you have not created an order yet.

In [50]:
orders = response.json()['orders']
len(orders)

20

## Ordering

In this example, we will order two `PSScene` `ortho_analytic_4b` images. For variations on this kind of order, see [Ordering Data](https://developers.planet.com/apis/orders/).

In this order, we request an `analytic` bundle. A bundle is a group of assets for an item. The `analytic_udm2` bundle for the  `PSScene` item contains 3 assets: the ortho-analytic image, the ortho-analytic xml file, and the ortho-udm2. See the [Scenes Product Bundles Reference](https://developers.planet.com/apis/orders/product-bundles-reference/) to learn about other bundles and other items.

### Place Order

In [51]:
# set content type to json
headers = {'content-type': 'application/json'}

In [52]:
request = {  
   "name":"simple order",
   "products":[
      {  
         "item_ids":[  
            "20151119_025740_0c74",
            "20151119_025741_0c74"
         ],
         "item_type":"PSScene",
         "product_bundle":"analytic_udm2"
      }
   ],
}

In [53]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

In [54]:
order_url = place_order(request, session.auth)

<Response [202]>
7555c25c-b090-4413-88c0-20ebe1df8952


### Cancel an Order

In [55]:
# report order state
requests.get(order_url, auth=session.auth).json()['state']

'running'

In [56]:
response = requests.put(order_url, auth=session.auth)
response

<Response [409]>

In [57]:
# report order state - it could take a little while to cancel
requests.get(order_url, auth=session.auth).json()['state']

'running'

### Poll for Order Success

In [58]:
# re-order since we canceled our last order
order_url = place_order(request, session.auth)

<Response [202]>
9c6dc5fe-8081-4998-9baa-e0308a01e4e4


In [59]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(order_url, auth)

queued
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
success


### View Results

In [62]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

In [63]:
[r['name'] for r in results]

['9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_udm2.tif',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_AnalyticMS_metadata.xml',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_AnalyticMS.tif',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_metadata.json',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025741_0c74_3B_udm2.tif',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025741_0c74_3B_AnalyticMS_metadata.xml',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025741_0c74_3B_AnalyticMS.tif',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025741_0c74_metadata.json',
 '9c6dc5fe-8081-4998-9baa-e0308a01e4e4/manifest.json']

## Download

### Downloading each asset individually

In [65]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [66]:
download_results(results)

9 items to download
downloading 9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_udm2.tif to data/9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_udm2.tif
downloading 9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_AnalyticMS_metadata.xml to data/9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_AnalyticMS_metadata.xml
downloading 9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_AnalyticMS.tif to data/9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_3B_AnalyticMS.tif
downloading 9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_metadata.json to data/9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025740_0c74_metadata.json
downloading 9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025741_0c74_3B_udm2.tif to data/9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSScene/20151119_025741_0c74_3B_udm2.tif
downloading 9c6dc5fe-8081-4998-9baa-e0308a01e4e4/PSSce

### Downloading as a single zip

To download all of the order assets as a single zip, the order request needs to be changed slightly with delivery instructions. After that, polling and downloading are the same.

In [67]:
zip_delivery = {"delivery": {"single_archive": True, "archive_type": "zip"}}
request_zip = request.copy()
request_zip.update(zip_delivery)
request_zip

{'name': 'simple order',
 'products': [{'item_ids': ['20151119_025740_0c74', '20151119_025741_0c74'],
   'item_type': 'PSScene',
   'product_bundle': 'analytic_udm2'}],
 'delivery': {'single_archive': True, 'archive_type': 'zip'}}

In [68]:
order_url = place_order(request_zip, auth=session.auth)

<Response [202]>
118c2da8-6512-41c4-a0fe-541bcfbe6285


In [69]:
poll_for_success(order_url, session.auth)

queued
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
success


In [70]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

In [71]:
download_results(results)

2 items to download
downloading 118c2da8-6512-41c4-a0fe-541bcfbe6285/output.zip to data/118c2da8-6512-41c4-a0fe-541bcfbe6285/output.zip
downloading 118c2da8-6512-41c4-a0fe-541bcfbe6285/manifest.json to data/118c2da8-6512-41c4-a0fe-541bcfbe6285/manifest.json
